<font size=6><b>Lec05. MNIST CNN

In [35]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# from keras.layers.convolutional import Conv2D, MaxPooling2D

from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.models import Sequential

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore')

from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)
pd.set_option('max_colwidth', None)



In [10]:
np.random.seed(1234)
tf.random.set_seed(1234)

# Data Load

In [13]:
(X_train, y_train), (X_test, y_test) = tensorflow.keras.datasets.mnist.load_data()

In [14]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

# EDA

# 전처리 & 가공
* 픽셀 (정규화)표준화  
* -------------------  StandardScaler (평균0 편차1 범위로 데이터 축소)
* -------------------  <b>MinMaxScaler (최소0 최대1 범위로 데이터 축소)</b>
* 이미지 크기 맞추기 ----------- NPL.padding
* 부족한 데이터 증강 ----------- SMOTE

## X  : 픽셀 (정규화)표준화

In [15]:
X_train_scaler = X_train / 255.0
X_test_scaler = X_test / 255.0

## y : 원핫인코딩

In [16]:
from tensorflow.keras.utils import to_categorical
y_train_oh = to_categorical(y_train,  num_classes=10, dtype='int32')
y_test_oh  = to_categorical(y_test,  num_classes=10, dtype='int32')

In [17]:
y_train[:5], y_train_oh[:5]

(array([5, 0, 4, 1, 9], dtype=uint8),
 array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]))

# CNN 모델

## 모델 생성

<pre>
tf.keras.layers.Conv2D(
    <font color=red><b>
    filters,
    kernel_size,
    strides=(1, 1),
    padding="valid",
    activation=None,
    input_shape=()
    </b></font>
    data_format=None,
    dilation_rate=(1, 1),
    groups=1,
    
    use_bias=True,
    bias_initializer="zeros",
    bias_regularizer=None,
    bias_constraint=None,
    
    kernel_initializer="glorot_uniform",
    kernel_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    
    **kwargs
)

tf.keras.layers.MaxPooling2D(
 <font color=red><b>
    pool_size=(2, 2), 
    strides=None, 
    padding="valid", 
    </b></font>
    data_format=None, **kwargs
)


<img src="https://lh3.googleusercontent.com/-yU9X-yhguE4/X7UZfhYhu9I/AAAAAAAAOSU/31OKn0P1hnIa59w_LwVCcuLDWyTrqvGZwCLcBGAsYHQ/w539-h288/image.png">

<img src="https://mblogthumb-phinf.pstatic.net/MjAxOTEyMjNfMTM2/MDAxNTc3MDg2MTU2MjAz.hGehbmIbYLZyyL6MCfKFepwr9lLFgi4NyNqJnMu7Yz4g.tY3SXH0lMdRNhMwsR45pGDzYj4HX2zdl5tkEgtS98mQg.PNG.seongcheol02/image.png?type=w800">

<pre>
# ------------------------------  Activation Map -------------------------------------
# 필터 Param : 필터(5*5)  * 채널(1or3) *  filters(16) + bias(16) = 416
#      X           W                                              + b               = y^
# (원본이미지)  : (필터) * 흑백(1) *  필터갯수(16)   + (각필터(보라색한장)당 바이어스) = Y^(합성곱)  -----> activation='relu'  == Activation Map
#   (28,28)    : 25        1          16개          + 16                = 416 
# ------------------------------------------------------------------------------------


In [27]:
model = Sequential()
# ------------------------------ CNN ----------------------------------
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu', input_shape=(28,28,1) ))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=24, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# ------------------------------ Full Connected == Affine --------------
model.add(Flatten())
model.add(Dense(units=32,  activation='relu'))         # ----------,  input_dim=784
model.add(Dense(units=16,  activation='relu')) 
model.add(Dense(units=10 , activation='softmax'))      #--------------- 멀티분류

model.compile(optimizer='adam',              
            loss='categorical_crossentropy',             #------원핫인코딩
            metrics=['accuracy'])              
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 24, 24, 16)        416       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 12, 12, 16)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 8, 8, 24)          9624      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 4, 24)         0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 384)               0         
                                                                 
 dense_9 (Dense)             (None, 32)               

## 학습

In [28]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

<pre>
tf.keras.callbacks.EarlyStopping(
    <font color=red><b>
    monitor="val_loss",
    patience=0,
    </b></font>
    min_delta=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

tf.keras.callbacks.ModelCheckpoint(
    <font color=red><b>
    filepath,
    monitor="val_loss",
    save_best_only=True,  #------------주의주의
    </b></font>
    verbose=0,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
    options=None,
    initial_value_threshold=None,
    **kwargs
)


In [32]:
my_stop = EarlyStopping(monitor="val_loss",patience=10)

my_point = ModelCheckpoint(
                  filepath = "./my_mnist_cnn_model/mnist_cnn_{epoch}_{val_loss:.5f}.h5"
                 , monitor="val_loss"
                 , save_best_only=True)

fit_res = model.fit(X_train_scaler, y_train_oh   #------X:sacling  y:onhot-encodin
                  , batch_size=10
                  , epochs=100
                  , validation_data=(X_test_scaler, y_test_oh)
                  , callbacks=[my_stop, my_point]
                 )

Epoch 1/100
6000/6000 [==============================] - 39s 6ms/step - loss: 0.1868 - accuracy: 0.9421 - val_loss: 0.0837 - val_accuracy: 0.9747
Epoch 2/100
6000/6000 [==============================] - 36s 6ms/step - loss: 0.0580 - accuracy: 0.9824 - val_loss: 0.0395 - val_accuracy: 0.9868
Epoch 3/100
6000/6000 [==============================] - 35s 6ms/step - loss: 0.0426 - accuracy: 0.9864 - val_loss: 0.0366 - val_accuracy: 0.9881
Epoch 4/100
6000/6000 [==============================] - 40s 7ms/step - loss: 0.0343 - accuracy: 0.9899 - val_loss: 0.0352 - val_accuracy: 0.9893
Epoch 5/100
6000/6000 [==============================] - 35s 6ms/step - loss: 0.0282 - accuracy: 0.9916 - val_loss: 0.0346 - val_accuracy: 0.9894
Epoch 6/100
6000/6000 [==============================] - 36s 6ms/step - loss: 0.0243 - accuracy: 0.9920 - val_loss: 0.0434 - val_accuracy: 0.9873
Epoch 7/100
6000/6000 [==============================] - 37s 6ms/step - loss: 0.0212 - accuracy: 0.9932 - val_loss: 0.0287 -

# 모델 불러오기
* ./my_mnist_cnn_model/mnist_cnn_7_0.028698207810521126.h5

* keras.models 모듈의 save_model(), load_model() 
<pre><font color=red><b>
  저장     : save_model(model_c, "mymodel.h5")
  불러오기 : a = load_model("mymodel.h5")

*./my_mnist_cnn_model/mnist_cnn_7_0.028698207810521126.h5

In [36]:
# save_model(model_c, "./mymodel.h5")
re_model = load_model("./my_mnist_cnn_model/mnist_cnn_7_0.028698207810521126.h5")
re_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 24, 24, 16)        416       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 12, 12, 16)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 8, 8, 24)          9624      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 4, 24)         0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 384)               0         
                                                                 
 dense_9 (Dense)             (None, 32)               

In [43]:
# re_model.get_weights()[:1]

## 평가
<pre>
* FC(5단)        : loss: 0.2910  accuracy : 0.947
* CNN(4단)-FC(2) : loss: 0.0286  accuracy : 0.993

In [44]:
loss, acc = re_model.evaluate(X_test_scaler, y_test_oh)
print(f"loss: {loss}  accuracy : {acc}")

313/313 [==============================] - 1s 4ms/step - loss: 0.0287 - accuracy: 0.9936
loss: 0.02869822084903717  accuracy : 0.9936000108718872


## myCNN vs. InceptionV3

<pre>
Total params: 23,058
Trainable params: 23,058
Non-trainable params: 0
__________________________

Total params: 23,851,784
Trainable params: 23,817,352
Non-trainable params: 34,432

# 잘 알려진 CNN 모델
* ref : https://keras.io/api/applications/

## InceptionV3

In [46]:
# from tensorflow.keras.applications import InceptionV3
# goomodel = InceptionV3()
# goomodel.summary()

<img src="https://production-media.paperswithcode.com/methods/inceptionv3onc--oview_vjAbOfw.png">

## Available models

<table width=500>
<thead>
<tr>
<th>Model</th>
<th align="right">Size (MB)</th>
<th align="right">Top-1 Accuracy</th>
<th align="right">Top-5 Accuracy</th>
<th align="right">Parameters</th>
<th align="right">Depth</th>
<th align="right">Time (ms) per inference step (CPU)</th>
<th align="right">Time (ms) per inference step (GPU)</th>
</tr>
</thead>
<tbody>
<tr>
<td><a href="xception">Xception</a></td>
<td align="right">88</td>
<td align="right">79.0%</td>
<td align="right">94.5%</td>
<td align="right">22.9M</td>
<td align="right">81</td>
<td align="right">109.4</td>
<td align="right">8.1</td>
</tr>
<tr>
<td><a href="vgg/#vgg16-function">VGG16</a></td>
<td align="right">528</td>
<td align="right">71.3%</td>
<td align="right">90.1%</td>
<td align="right">138.4M</td>
<td align="right">16</td>
<td align="right">69.5</td>
<td align="right">4.2</td>
</tr>
<tr>
<td><a href="vgg/#vgg19-function">VGG19</a></td>
<td align="right">549</td>
<td align="right">71.3%</td>
<td align="right">90.0%</td>
<td align="right">143.7M</td>
<td align="right">19</td>
<td align="right">84.8</td>
<td align="right">4.4</td>
</tr>
<tr>
<td><a href="resnet/#resnet50-function">ResNet50</a></td>
<td align="right">98</td>
<td align="right">74.9%</td>
<td align="right">92.1%</td>
<td align="right">25.6M</td>
<td align="right">107</td>
<td align="right">58.2</td>
<td align="right">4.6</td>
</tr>
<tr>
<td><a href="resnet/#resnet50v2-function">ResNet50V2</a></td>
<td align="right">98</td>
<td align="right">76.0%</td>
<td align="right">93.0%</td>
<td align="right">25.6M</td>
<td align="right">103</td>
<td align="right">45.6</td>
<td align="right">4.4</td>
</tr>
<tr>
<td><a href="resnet/#resnet101-function">ResNet101</a></td>
<td align="right">171</td>
<td align="right">76.4%</td>
<td align="right">92.8%</td>
<td align="right">44.7M</td>
<td align="right">209</td>
<td align="right">89.6</td>
<td align="right">5.2</td>
</tr>
<tr>
<td><a href="resnet/#resnet101v2-function">ResNet101V2</a></td>
<td align="right">171</td>
<td align="right">77.2%</td>
<td align="right">93.8%</td>
<td align="right">44.7M</td>
<td align="right">205</td>
<td align="right">72.7</td>
<td align="right">5.4</td>
</tr>
<tr>
<td><a href="resnet/#resnet152-function">ResNet152</a></td>
<td align="right">232</td>
<td align="right">76.6%</td>
<td align="right">93.1%</td>
<td align="right">60.4M</td>
<td align="right">311</td>
<td align="right">127.4</td>
<td align="right">6.5</td>
</tr>
<tr>
<td><a href="resnet/#resnet152v2-function">ResNet152V2</a></td>
<td align="right">232</td>
<td align="right">78.0%</td>
<td align="right">94.2%</td>
<td align="right">60.4M</td>
<td align="right">307</td>
<td align="right">107.5</td>
<td align="right">6.6</td>
</tr>
<tr>
<td><a href="inceptionv3">InceptionV3</a></td>
<td align="right">92</td>
<td align="right">77.9%</td>
<td align="right">93.7%</td>
<td align="right">23.9M</td>
<td align="right">189</td>
<td align="right">42.2</td>
<td align="right">6.9</td>
</tr>
<tr>
<td><a href="inceptionresnetv2">InceptionResNetV2</a></td>
<td align="right">215</td>
<td align="right">80.3%</td>
<td align="right">95.3%</td>
<td align="right">55.9M</td>
<td align="right">449</td>
<td align="right">130.2</td>
<td align="right">10.0</td>
</tr>
<tr>
<td><a href="mobilenet">MobileNet</a></td>
<td align="right">16</td>
<td align="right">70.4%</td>
<td align="right">89.5%</td>
<td align="right">4.3M</td>
<td align="right">55</td>
<td align="right">22.6</td>
<td align="right">3.4</td>
</tr>
<tr>
<td><a href="mobilenet/#mobilenetv2-function">MobileNetV2</a></td>
<td align="right">14</td>
<td align="right">71.3%</td>
<td align="right">90.1%</td>
<td align="right">3.5M</td>
<td align="right">105</td>
<td align="right">25.9</td>
<td align="right">3.8</td>
</tr>
<tr>
<td><a href="densenet/#densenet121-function">DenseNet121</a></td>
<td align="right">33</td>
<td align="right">75.0%</td>
<td align="right">92.3%</td>
<td align="right">8.1M</td>
<td align="right">242</td>
<td align="right">77.1</td>
<td align="right">5.4</td>
</tr>
<tr>
<td><a href="densenet/#densenet169-function">DenseNet169</a></td>
<td align="right">57</td>
<td align="right">76.2%</td>
<td align="right">93.2%</td>
<td align="right">14.3M</td>
<td align="right">338</td>
<td align="right">96.4</td>
<td align="right">6.3</td>
</tr>
<tr>
<td><a href="densenet/#densenet201-function">DenseNet201</a></td>
<td align="right">80</td>
<td align="right">77.3%</td>
<td align="right">93.6%</td>
<td align="right">20.2M</td>
<td align="right">402</td>
<td align="right">127.2</td>
<td align="right">6.7</td>
</tr>
<tr>
<td><a href="nasnet/#nasnetmobile-function">NASNetMobile</a></td>
<td align="right">23</td>
<td align="right">74.4%</td>
<td align="right">91.9%</td>
<td align="right">5.3M</td>
<td align="right">389</td>
<td align="right">27.0</td>
<td align="right">6.7</td>
</tr>
<tr>
<td><a href="nasnet/#nasnetlarge-function">NASNetLarge</a></td>
<td align="right">343</td>
<td align="right">82.5%</td>
<td align="right">96.0%</td>
<td align="right">88.9M</td>
<td align="right">533</td>
<td align="right">344.5</td>
<td align="right">20.0</td>
</tr>
<tr>
<td><a href="efficientnet/#efficientnetb0-function">EfficientNetB0</a></td>
<td align="right">29</td>
<td align="right">77.1%</td>
<td align="right">93.3%</td>
<td align="right">5.3M</td>
<td align="right">132</td>
<td align="right">46.0</td>
<td align="right">4.9</td>
</tr>
<tr>
<td><a href="efficientnet/#efficientnetb1-function">EfficientNetB1</a></td>
<td align="right">31</td>
<td align="right">79.1%</td>
<td align="right">94.4%</td>
<td align="right">7.9M</td>
<td align="right">186</td>
<td align="right">60.2</td>
<td align="right">5.6</td>
</tr>
<tr>
<td><a href="efficientnet/#efficientnetb2-function">EfficientNetB2</a></td>
<td align="right">36</td>
<td align="right">80.1%</td>
<td align="right">94.9%</td>
<td align="right">9.2M</td>
<td align="right">186</td>
<td align="right">80.8</td>
<td align="right">6.5</td>
</tr>
<tr>
<td><a href="efficientnet/#efficientnetb3-function">EfficientNetB3</a></td>
<td align="right">48</td>
<td align="right">81.6%</td>
<td align="right">95.7%</td>
<td align="right">12.3M</td>
<td align="right">210</td>
<td align="right">140.0</td>
<td align="right">8.8</td>
</tr>
<tr>
<td><a href="efficientnet/#efficientnetb4-function">EfficientNetB4</a></td>
<td align="right">75</td>
<td align="right">82.9%</td>
<td align="right">96.4%</td>
<td align="right">19.5M</td>
<td align="right">258</td>
<td align="right">308.3</td>
<td align="right">15.1</td>
</tr>
<tr>
<td><a href="efficientnet/#efficientnetb5-function">EfficientNetB5</a></td>
<td align="right">118</td>
<td align="right">83.6%</td>
<td align="right">96.7%</td>
<td align="right">30.6M</td>
<td align="right">312</td>
<td align="right">579.2</td>
<td align="right">25.3</td>
</tr>
<tr>
<td><a href="efficientnet/#efficientnetb6-function">EfficientNetB6</a></td>
<td align="right">166</td>
<td align="right">84.0%</td>
<td align="right">96.8%</td>
<td align="right">43.3M</td>
<td align="right">360</td>
<td align="right">958.1</td>
<td align="right">40.4</td>
</tr>
<tr>
<td><a href="efficientnet/#efficientnetb7-function">EfficientNetB7</a></td>
<td align="right">256</td>
<td align="right">84.3%</td>
<td align="right">97.0%</td>
<td align="right">66.7M</td>
<td align="right">438</td>
<td align="right">1578.9</td>
<td align="right">61.6</td>
</tr>
<tr>
<td><a href="efficientnet_v2/#efficientnetv2b0-function">EfficientNetV2B0</a></td>
<td align="right">29</td>
<td align="right">78.7%</td>
<td align="right">94.3%</td>
<td align="right">7.2M</td>
<td align="right">-</td>
<td align="right">-</td>
<td align="right">-</td>
</tr>
<tr>
<td><a href="efficientnet_v2/#efficientnetv2b1-function">EfficientNetV2B1</a></td>
<td align="right">34</td>
<td align="right">79.8%</td>
<td align="right">95.0%</td>
<td align="right">8.2M</td>
<td align="right">-</td>
<td align="right">-</td>
<td align="right">-</td>
</tr>
<tr>
<td><a href="efficientnet_v2/#efficientnetv2b2-function">EfficientNetV2B2</a></td>
<td align="right">42</td>
<td align="right">80.5%</td>
<td align="right">95.1%</td>
<td align="right">10.2M</td>
<td align="right">-</td>
<td align="right">-</td>
<td align="right">-</td>
</tr>
<tr>
<td><a href="efficientnet_v2/#efficientnetv2b3-function">EfficientNetV2B3</a></td>
<td align="right">59</td>
<td align="right">82.0%</td>
<td align="right">95.8%</td>
<td align="right">14.5M</td>
<td align="right">-</td>
<td align="right">-</td>
<td align="right">-</td>
</tr>
<tr>
<td><a href="efficientnet_v2/#efficientnetv2s-function">EfficientNetV2S</a></td>
<td align="right">88</td>
<td align="right">83.9%</td>
<td align="right">96.7%</td>
<td align="right">21.6M</td>
<td align="right">-</td>
<td align="right">-</td>
<td align="right">-</td>
</tr>
<tr>
<td><a href="efficientnet_v2/#efficientnetv2m-function">EfficientNetV2M</a></td>
<td align="right">220</td>
<td align="right">85.3%</td>
<td align="right">97.4%</td>
<td align="right">54.4M</td>
<td align="right">-</td>
<td align="right">-</td>
<td align="right">-</td>
</tr>
<tr>
<td><a href="efficientnet_v2/#efficientnetv2l-function">EfficientNetV2L</a></td>
<td align="right">479</td>
<td align="right">85.7%</td>
<td align="right">97.5%</td>
<td align="right">119.0M</td>
<td align="right">-</td>
<td align="right">-</td>
<td align="right">-</td>
</tr>
</tbody>
</table>